In [ ]:
from sklearn import datasets
import numpy as np
import persim
import persim.plot
import ripser
import matplotlib.pyplot as plt

In [ ]:
data_clean = datasets.make_circles()[0]
data_noisy = datasets.make_circles(noise=0.1)[0]

In [ ]:
plt.scatter(data_clean[:,0], data_clean[:,1], label="clean data")
plt.scatter(data_noisy[:,0], data_noisy[:,1], label="noisy data")

## Generate $H_1$ diagrams for each of the data sets

In [ ]:
dgm_clean = ripser.ripser(data_clean)['dgms'][1]
dgm_noisy = ripser.ripser(data_noisy)['dgms'][1]

In [ ]:
ripser.plot_dgms([dgm_clean, dgm_noisy] , labels=['Clean $H_1$', 'Noisy $H_1$'])

## Compute and visualize Bottleneck distance

**bottleneck distance**: Bottleneck distance measures the similarity between two persistence diagrams. It's the shortest distance _b_ for which there exists a perfect matching between the points of the two diagrams (+ all the diagonal points) such that any couple of matched points are at distance at most _b_.   

The `bottleneck` function has the option of returning the matching when the parameter `matching` is set to `True`. With the returned data, we can use the `plot.bottleneck_matching` function to visualize which persistence points contributed to the distance.

In [ ]:
distance_bottleneck, (matching, D) = persim.bottleneck(dgm_clean, dgm_noisy, matching=True)

In [ ]:
persim.plot.bottleneck_matching(dgm_clean, dgm_noisy, matching, D, labels=['Clean $H_1$', 'Noisy $H_1$'])

The default option of `matching=False` will return just the distance if that is all you're interested in.

In [ ]:
persim.bottleneck(dgm_clean, dgm_noisy)

In [ ]:
dgm1 = np.array([
    [0.5, 1],
    [0.6, 1.1]
])
dgm2 = np.array([
    [0.5, 1.1],
#     [0.7,0.8]
])

d, (matching, D) = persim.bottleneck(
    dgm1,
    dgm2,
    matching=True
)

persim.plot.bottleneck_matching(dgm1, dgm2, matching, D, labels=['Clean $H_1$', 'Noisy $H_1$'])
plt.title(f"Distance {d:.3f}")

In [ ]:
matching, D

## Sliced Wasserstein distance


Sliced Wasserstein Kernels for persistence diagrams were introduced by [Carriere et al, 2017](https://arxiv.org/abs/1706.03358) and implemented by Alice Patania.

The general idea is to compute an approximation of the Wasserstein distance by computing the distance in 1-dimension repeatedly, and use the results as measure. To do so, the points of each persistence diagram are projected onto M lines that pass through (0,0) and forms an angle theta with the axis x.

In [ ]:
persim.sliced_wasserstein(dgm_clean, dgm_noisy)

The parameter `M` controls the number of iterations to run

In [ ]:
Ms = range(5, 100, 2)
ds = [persim.sliced_wasserstein(dgm_clean, dgm_noisy, M=M) for M in Ms]

In [ ]:
plt.plot(Ms, ds)
plt.xlabel("M Iterations")
plt.ylabel("Approximate Distance")
plt.title("Relationship between M and distance")
plt.show()

## Heat Kernel Distance

We also implement the heat kernel distance

In [ ]:
persim.heat(dgm_clean, dgm_noisy)

The parameter `sigma` controls the heat diffusion.

In [ ]:
sigmas = np.linspace(0.1, 10, 100)
ds = [persim.heat(dgm_clean, dgm_noisy, sigma=s) for s in sigmas]

In [ ]:
plt.plot(sigmas, ds)
plt.xlabel("Heat diffusion parameter")
plt.ylabel("Approximate Distance")
plt.title("Relationship between sigma and distance")
plt.show()

Note: the heat kernel is quite slow in general, but also one of the most discriminative (out of experience). 

# Exercise
- measure the distance between tori of various dimensions, sampling density etc...
- generate a few different persistent diagrams (anyway you want) and compare how well the distances work?

In [ ]:
# hint for first exercise: import stuff from the previous notebooks (duh..)
# help for the second exercise: use data in '../data/ToyData_PD_TextFiles/'
# obtained from here:
# https://github.com/CSU-TDA/PersistenceImages/tree/master/matlab_code/sixShapeClasses/ToyData_PD_TextFiles

In [ ]:
# solution to the second exercise:
import os
pd_data = {}
data_dir = '../data/ToyData_PD_TextFiles/'

for f in os.listdir(data_dir):
    pd_data[f] = np.genfromtxt(data_dir + f);
    

In [ ]:
import time 
from itertools import combinations
now = time.time();
h = np.zeros((10,10));
keys = np.array([*pd_data])[list(np.random.randint(0,high=600,size=10))];

for i,j in combinations(range(len(keys)),2):
    h[i,j] = persim.heat(pd_data[keys[i]], pd_data[keys[j]])
print(time.time() - now)

In [ ]:
plt.imshow(h+h.T)
plt.colorbar()

# Exercises

In [ ]:
- Can you cluster 